In [1]:
from langchain_community.document_loaders import TextLoader

In [5]:
local_path = "files/company.txt"

loader = TextLoader(local_path, encoding = 'UTF-8')
loader.load()
pages = loader.load_and_split()

In [6]:
pages[0].page_content

'Зеленые Поляны Агроферма - это современное сельскохозяйственное предприятие, которое специализируется на выращивании органических овощей и зеленых культур. Наша агроферма расположена в питательной почве Поднебесной Долины, известной своим благоприятным климатом и чистым экологическим окружением.'

In [7]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma

In [8]:
# Split and chunk 
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2500, chunk_overlap=100)
chunks = text_splitter.split_documents(pages)

In [9]:
# Add to vector database
vector_db = Chroma.from_documents(
    documents=chunks, 
    embedding=OllamaEmbeddings(model="nomic-embed-text",show_progress=True),
    collection_name="local-rag"
)

OllamaEmbeddings: 100%|██████████| 1/1 [00:04<00:00,  4.07s/it]


In [10]:
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever

In [11]:
# LLM from Ollama
local_model = "llama3:8b"
llm = ChatOllama(model=local_model)

In [12]:
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""Вы - ассистент по языковой модели искусственного интеллекта для ответа по документу.
 Не нужно придумывать своих условий или что-то подобное, только что представленно в тексте.
 Ваша задача - найти информацию из текста документа условия участия и только их.
 Никакой другой или вводной информации не надо. 
 Отвечай только на русском языке для этого.
 Исходный вопрос: {question}""",
)

In [13]:
retriever = MultiQueryRetriever.from_llm(
    vector_db.as_retriever(), 
    llm,
    prompt=QUERY_PROMPT
)

# RAG prompt
template = """Отвечай на вопросы без вводной инфформации согласно контексту.  Только список. В конце переведи свой ответ на русский язык.:
{context}
Вопрос: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [14]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [15]:
respoonse = chain.invoke("Какое название у предприятия? И отрасль деятельности?")
respoonse

OllamaEmbeddings: 100%|██████████| 1/1 [00:04<00:00,  4.19s/it]
Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1


'Here is the answer:\n\n**Green Poles Agricultural Farm**\n**Organic vegetable and green culture farming**\n\nTranslation to Russian:\n**Зеленые Поляны Агроферма**\n**Органические овощеводство и культура зелёных культур**'

In [16]:
import torch
import nltk
from sacrebleu.metrics import BLEU

from rouge import Rouge
rouge_scorer = Rouge()

In [17]:
def evaluate_response(reference, response):
    # BLEU оценка
    bleu_scorer = BLEU(effective_order=True)
    blue_score = bleu_scorer.sentence_score(
        hypothesis=response,
        references=[reference],
    )

    # ROUGE оценка
    score = rouge_scorer.get_scores(
        hyps=response,
        refs=reference,
    )
    rouge_score = score[0]["rouge-l"]["f"]


    return blue_score, rouge_score

In [18]:
reference_text = '''Зеленые Поляны Агроферма, отрасль деятельности - сельскохозяйственное предприятие'''

In [19]:
bleu, rouge = evaluate_response(reference_text, respoonse)

In [20]:
print("BLEU:", bleu.score/100)
print("ROUGE:", rouge)

BLEU: 0.030977170338913472
ROUGE: 0.058823525813149015
